# Importações

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from joblib import load

In [ ]:
model = load('models/rf_best_model.joblib')

train_df = pd.read_csv('data/train_data.csv')
test_df = pd.read_csv('data/test_data.csv')

target = 'Kingdom_group_encoded'
labels = ['Animalia', 'Bacteria', 'Plantae', 'Virus']

X_train = train_df.drop(target, axis=1)
y_train = train_df[target]

X_test = test_df.drop(target, axis=1)
y_test = test_df[target]